In [7]:
from datasets import load_dataset
import requests
import json
from supabase import create_client, Client
import time
import os
import openai
import dotenv
dotenv.load_dotenv()
openai_api_key = os.environ['OPENAI_API_KEY']
supabase_url = os.environ['SUPABASE_URL']
supabase_api_key = os.environ['SUPABASE_API_KEY'] 
print(supabase_url)

https://zioyzzneyeglzxlgtdrw.supabase.co


In [15]:
# Load the dataset, initializa supabase client
model_name = "text-embedding-3-small"
dataset = load_dataset("MediaTek-Research/TCEval-v2", "drcd")
supabase: Client = create_client(supabase_url, supabase_api_key)

# 若已經處理過，則刪除後重新來一次
# supabase.table('drcd_questions').delete().eq('model', model_name).execute()
# supabase.table('drcd_paragraphs').delete().eq('model', model_name).execute()


在運行以下內容，建立embeddings之前，我們應該要建立資料表。我建好資料表就可以了，大家可以直接用。
```sql
create table drcd_paragraphs (
  id bigint primary key generated always as identity,
  content text,
  embedding vector(1536),
  model text
);

create table drcd_questions (
  id bigint primary key generated always as identity,
  dataset_id integer,
  content text,
  embedding vector(1536),
  model text,
  paragraph_id bigint references drcd_paragraphs(id)
);
```

In [16]:
dataset["test"][100]

{'id': 'test-100',
 'paragraph': '詳細規劃的內容有確定道路紅線、道路斷面、小區範圍、街坊及專用地段主要特別的坐標和標高。確定居住建築、公共建築、公共綠地、道路廣場這些項目的具體位置和用地。確定工業、倉儲等項目的具體位置和用地。綜合安排專用地段和各項工程管線、工程構築物的位置和用地。主要幹道和廣場建築群的平面、立面規劃設計。都市高速公路、城際高速公路、其他的快速道路。在中國，因為行政管理體制的區別，公路是由交通運輸部來管理的，城市道路由建設部門來管理。公路按技術標準分為高速公路、一級公路、二級公路、三級公路、四級公路。幹線道路不是都市主要的骨幹，而是作為連接鄰近住宅區及外圍地帶的道路。區劃道路供給住宅地使用的道路。特殊道路主要是為了汽車以外的交通而供用的道路。',
 'question': '公路在中國是由什麼單位負責的？',
 'references': ['交通運輸部']}

In [ ]:
def get_embeddings(input, model):
  payload = { "input": input, "model": model }
  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/embeddings', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["data"][0]["embedding"]
  else:
    time.sleep(3)
    print("embedding error..... retrying")
    # retry
    return get_embeddings(input, model)


In [17]:
# 時間：59分46秒
# 金錢：0.02美元

current_paragraph = ''
current_paragraph_id = 0

for i in range(0, 3493):
    if i%10 == 0:
      print(i)
    data = dataset["test"][i]
    #paragraph總共有1000段文檔
    if current_paragraph != data["paragraph"]:
      embedding = get_embeddings( data["paragraph"], model_name )
      response = supabase.table('drcd_paragraphs').upsert({"content": data["paragraph"], "embedding": embedding, "model": model_name }).execute()
      current_paragraph = data["paragraph"]
      current_paragraph_id = response.data[0]["id"]

    q_embedding = get_embeddings( data["question"], model_name )
    supabase.table('drcd_questions').upsert({"dataset_id": i, "content": data["question"], "embedding": q_embedding, "model": model_name, "paragraph_id": current_paragraph_id }).execute()

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2